<a href="https://colab.research.google.com/github/h0806449f/Pytorch/blob/main/02_neural_network_clf_fromYTvideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.youtube.com/watch?v=V_xro1bcAuA&t=13186s
# 請從8:57:57 開始

SyntaxError: ignored